In [16]:
import cv2
import numpy as np
import os
import mediapipe as mp

In [17]:
# 初始化 Mediapipe Pose 模型
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [18]:
def mediapipe_detection(image, model):
    """
    This function detects human pose estimation keypoints from webcam footage
    
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [19]:
def draw_landmarks(image, results):
    """
    This function draws keypoints and landmarks detected by the human pose estimation model
    
    """
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )

In [20]:
def extract_keypoints(results):
    """
    Processes and organizes the keypoints detected from the pose estimation model 
    to be used as inputs for the exercise decoder models
    
    """
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose

In [21]:
# 路径和参数设置（保持不变）
DATA_PATH = os.path.join(os.getcwd(),'data')
actions = np.array(['curl', 'press', 'squat'])
colors = [(245,117,16), (117,245,16), (16,117,245)]
no_sequences = 5
start_folder = 101

In [22]:
# 设置每个视频序列的帧数
sequence_length = 30  # 假设每个视频序列包含 30 帧

In [23]:
# 创建数据文件夹
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

for action in actions:
    dir_path = os.path.join(DATA_PATH, action)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    for sequence in range(no_sequences):
        sequence_path = os.path.join(dir_path, str(sequence))
        if not os.path.exists(sequence_path):
            os.makedirs(sequence_path)

In [24]:
video_files = {'curl': 'videos/curls.mp4', 'press': 'videos/press.mp4', 'squat': 'videos/squats.mp4'}  # 替换为实际视频路径


In [25]:
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for action, video_file in video_files.items():
        cap = cv2.VideoCapture(video_file)
        frame_count = 0
        sequence = 0

        while cap.isOpened() and sequence < no_sequences:
            ret, frame = cap.read()
            if not ret:
                break
            
            image, results = mediapipe_detection(frame, pose)
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_count))
            np.save(npy_path, keypoints)

            frame_count += 1
            if frame_count == sequence_length:
                sequence += 1
                frame_count = 0

        cap.release()

I0000 00:00:1707165639.581517       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 76.3), renderer: Apple M1


processing data

In [26]:
label_map = {label:num for num, label in enumerate(actions)}

In [27]:
sequences, labels = [], []
for action in actions:
    # 获取动作文件夹的路径
    action_path = os.path.join(DATA_PATH, action)
    
    # 过滤掉非数字的文件夹名（排除如 .DS_Store 的文件）
    sequence_dirs = [dir_name for dir_name in os.listdir(action_path) if dir_name.isdigit()]

    # 现在可以安全地将目录名转换为整数
    for sequence in np.array(sequence_dirs).astype(int):
        window = []
        for frame_num in range(sequence_length):         
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)  
            
        sequences.append(window)
        labels.append(label_map[action])





In [28]:
# 检查 label_map 是否包含所有动作
print("Label Map:", label_map)

# 在加载数据后，检查 labels 列表
print("Labels Sample:", labels[:15])  # 打印前10个标签作为样本

# 检查 labels 列表的长度是否与数据点总数相匹配
print("Total number of labels:", len(labels))


Label Map: {'curl': 0, 'press': 1, 'squat': 2}
Labels Sample: [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]
Total number of labels: 15


In [29]:
# 计算每个类别的样本数量
unique, counts = np.unique(labels, return_counts=True)
class_counts = dict(zip(unique, counts))

print("Class counts:", class_counts)


Class counts: {0: 5, 1: 5, 2: 5}


In [30]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [71]:

labels_array = np.array(labels).reshape(-1, 1)
encoder = OneHotEncoder()
y = encoder.fit_transform(labels_array).toarray()  # 使用 toarray() 转换为密集矩阵
X = np.array(sequences)
print("Shape of encoded labels:", y.shape)



Shape of encoded labels: (15, 3)


In [72]:
# 分割数据集为训练集、验证集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=15/90, random_state=2)

# 输出数据集的形状以确认
print("Training set:", X_train.shape, y_train.shape)
#print("Validation set:", X_val.shape, y_val.shape)
print("Testing set:", X_test.shape, y_test.shape)

Training set: (10, 30, 132) (10, 3)
Testing set: (5, 30, 132) (5, 3)


In [73]:
unique_labels_train = np.unique(y_train)
unique_labels_test = np.unique(y_test)
print("Unique labels in training data:", unique_labels_train)
print("Unique labels in test data:", unique_labels_test)
y_train


Unique labels in training data: [0. 1.]
Unique labels in test data: [0. 1.]


array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [74]:
import torch
import torch.nn as nn
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [75]:
tensor_X_test = torch.from_numpy(X_test)
print('test_data_size:', tensor_X_test.size())
tensor_X_test = torch.from_numpy(X_test)
print('test_data_size:',tensor_X_test.size())
print(y_test)


tensor_y_test = torch.from_numpy(y_test)
print('test_label_size:',tensor_y_test.size())
n_data_size_test = tensor_X_test.size()[0]



print('n_data_size_test:',n_data_size_test)

tensor_X_train = torch.from_numpy(X_train)
print('train_data_size:',tensor_X_train.size())
tensor_y_train = torch.from_numpy(y_train)
print('train_label_size:',tensor_y_train.size())
n_data_size_train = tensor_X_train.size()[0]
print('n_data_size_train:',n_data_size_train)

test_data_size: torch.Size([5, 30, 132])
test_data_size: torch.Size([5, 30, 132])
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
test_label_size: torch.Size([5, 3])
n_data_size_test: 5
train_data_size: torch.Size([10, 30, 132])
train_label_size: torch.Size([10, 3])
n_data_size_train: 10


In [76]:
tensor_y_train

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.]], dtype=torch.float64)

In [77]:
import numpy as np

# 假设您的标签数组是 tensor_y_train
unique_labels = np.unique(tensor_y_train.numpy())
print("Unique labels in the dataset:", unique_labels)


Unique labels in the dataset: [0. 1.]


In [78]:
class LSTM(nn.Module):
    
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super(LSTM,self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim,hidden_dim,layer_num,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        self.bn = nn.BatchNorm1d(30)
        
    def forward(self,inputs):
        x = self.bn(inputs)
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [79]:
n_hidden = 128  # 隐藏层维度
n_joints = 132  # 输入维度，每帧的特征数量
n_categories = 3  # 输出维度，类别数量
n_layer = 3  # LSTM 层数
rnn = LSTM(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM(
  (lstm): LSTM(132, 128, num_layers=3, batch_first=True)
  (fc): Linear(in_features=128, out_features=3, bias=True)
  (bn): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [80]:
LABELS = ['curls','press','squats']


In [81]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return LABELS[category_i], category_i

In [86]:
import random
def randomTrainingExampleBatch(batch_size, flag, num=-1):
    if flag == 'train':
        X = tensor_X_train
        y = tensor_y_train
        data_size = n_data_size_train
    elif flag == 'test':
        X = tensor_X_test
        y = tensor_y_test
        data_size = n_data_size_test
    
    # 确保不会尝试从小于批量大小的数据集中提取数据
    if data_size < batch_size:
        raise ValueError("Data size is smaller than batch size")
    
    if num == -1:
        ran_num = random.randint(0, data_size - batch_size)
    else:
        ran_num = num
    
    # print("ran_num_2: %d, %d, %d" % (ran_num, data_size - batch_size, data_size))
    pose_sequence_tensor = X[ran_num:(ran_num + batch_size)]
    category_tensor = y[ran_num:ran_num + batch_size, :]
    return category_tensor.long(), pose_sequence_tensor


In [83]:
import time
import math
import torch.optim as optim


In [84]:
# 训练循环
learning_rate = 0.0005
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)
criterion = nn.CrossEntropyLoss()

current_loss = 0
all_losses = []



def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

In [87]:
n_iters = 20000
print_every = 1000
plot_every = 1000
batch_size = 3

for iter in range(1, n_iters + 1):
    category_tensor, input_sequence = randomTrainingExampleBatch(batch_size, 'train')
    input_sequence = input_sequence.to(device)
    category_tensor = category_tensor.to(device)

    # 获取类别索引
    _, category_tensor = category_tensor.max(dim=1)

    optimizer.zero_grad()

    output = rnn(input_sequence.float())
    loss = criterion(output, category_tensor)
    loss.backward()
    optimizer.step()

    current_loss += loss.item()

    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == LABELS[category_tensor[0]] else '✗ (%s)' % LABELS[category_tensor[0]]
        print('%d %d%% (%s) %.4f  / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

1000 5% (0m 40s) 0.0900  / press ✓
2000 10% (0m 56s) 0.0141  / press ✓
3000 15% (1m 12s) 0.0093  / curls ✓
4000 20% (1m 31s) 0.0050  / squats ✓
5000 25% (1m 47s) 0.0037  / squats ✓
6000 30% (2m 4s) 0.0034  / curls ✓
7000 35% (2m 20s) 0.0023  / squats ✓
8000 40% (2m 40s) 0.0025  / curls ✓
9000 45% (2m 56s) 0.0022  / curls ✓
10000 50% (3m 12s) 0.0015  / squats ✓
11000 55% (3m 29s) 0.0013  / squats ✓
12000 60% (3m 46s) 0.0012  / press ✓
13000 65% (4m 1s) 0.0011  / press ✓
14000 70% (4m 17s) 0.0012  / press ✓
15000 75% (4m 34s) 0.0011  / curls ✓
16000 80% (4m 51s) 0.0011  / curls ✓
17000 85% (5m 8s) 0.0008  / press ✓
18000 90% (5m 26s) 0.0009  / curls ✓
19000 95% (5m 42s) 0.0008  / squats ✓
20000 100% (5m 59s) 0.0006  / squats ✓


In [ ]:
torch.save(rnn.state_dict(),'lstm.pkl')


In [ ]:
def test(flag):
    if flag == 'train':
        n = n_data_size_train
    elif flag == 'test':
        n = n_data_size_test

    with torch.no_grad():
        right = 0
        start = time.time()
        for i in range(n):
            category_tensor, inputs = randomTrainingExampleBatch(1, flag, i)
            category_index = category_tensor[0].nonzero(as_tuple=True)[0].item()
            category = LABELS[category_index]

            inputs = inputs.to(device).float()
            output = rnn(inputs)
            
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess_i == category_index else '✗ (%s)' % category
            print('%d %d%% (%s) %.4f / %s %s' % (i + 1, (i + 1) / n * 100, timeSince(start), output[0][guess_i], guess, correct))
            
            if category_index == guess_i:
                right += 1

    print(flag, 'accuracy', right / n)


In [ ]:
# def test(flag):
#     if flag == 'train':
#         n = n_data_size_train
#     elif flag == 'test':
#         n = n_data_size_test

#     with torch.no_grad():
#         right = 0
#         for i in range(n):
#             category_tensor, inputs = randomTrainingExampleBatch(1, flag, i)
#             category_index = category_tensor[0].nonzero(as_tuple=True)[0].item()
#             category = LABELS[category_index]

#             # 确保 inputs 在正确的设备上，并且是正确的数据类型
#             inputs = inputs.to(device).float()
#             output = rnn(inputs)
            
#             guess, guess_i = categoryFromOutput(output)
#             if category_index == guess_i:
#                 right += 1

#     print(flag, 'accuracy', right / n)


In [ ]:
test(flag='test')


1 20% (0m 0s) 5.0628 / curls ✓
2 40% (0m 0s) 5.0896 / press ✓
3 60% (0m 0s) 5.0983 / press ✓
4 80% (0m 0s) 5.0645 / curls ✓
5 100% (0m 0s) 5.3155 / squats ✓
test accuracy 1.0


In [ ]:
test(flag='train')


1 10% (0m 0s) 5.0680 / curls ✓
2 20% (0m 0s) 5.0733 / curls ✓
3 30% (0m 0s) 5.6990 / squats ✓
4 40% (0m 0s) 5.0662 / curls ✓
5 50% (0m 0s) 5.7036 / squats ✓
6 60% (0m 0s) 5.0956 / press ✓
7 70% (0m 0s) 5.0793 / press ✓
8 80% (0m 0s) 5.7000 / squats ✓
9 90% (0m 0s) 5.6773 / squats ✓
10 100% (0m 0s) 5.0941 / press ✓
train accuracy 1.0
